# Exercise 19

## Audio Klassifizierung

Wir möchten nun versuchen Audio Dateien zu klassifizieren. In diesem Fall haben wir Geräusche von Hunden und Katzen. Können wir erkennen von welchem Tier das Geräusch stammt?

In [ ]:
import os
import numpy as np
from scipy import signal
import scipy.io.wavfile as wavf
import matplotlib.pyplot as plt

# Diese Funktion erstellt einen Ton von Länge s Sekunden und mit Frequenz hz.
# Die samplerate gibt an wie viele "Werte" wir pro Sekunde abspielen.
# Die Werte sind einfach eine Liste von Zahlen!
def create_sound_wave(hz, s, samplerate=44100):
    return [np.sin(i * 2 * np.pi * hz / samplerate) for i in range(int(samplerate * s))]

# Diese Funktion nimmt nun die Zahlenwerte und speichert sie als wav file.
def save_sound(sound_list, samplerate=44100, file='out.wav'):
    wavf.write(out_f, samplerate, np.array(sound_list, dtype=np.float32))

# Dise Funktion spielt schliesslich das wav file ab.
def play_sound(file='out.wav'):
    os.system(f'aplay {file}')

# List eine wav Datei und gibt ein array mit den Werten zurück.
def read_sound(file='out.wav'):
    return wavf.read(file)[1]

# Erstellt ein Spektrogram, wobei f die Frequenzen, t die Zeit und Sxx die Werte sind.
def create_spectrogram(sound_wave, samplerate=44100):
    f, t, Sxx = signal.spectrogram(sound_wave, samplerate)
    return f, t, Sxx

# Erstellt ein Spektrogram Plot.
def plot_spectrogram(sound_wave, samplerate=44100):
    f, t, Sxx = create_spectrogram(sound_wave, samplerate)
    
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

### Aufgabe 1

Spiele zuerst ein paar Audiofiles aus dem `data` Ordner ab,

In [ ]:
play_sound(file='data/test/cat_75.wav')

### Aufgabe 2

Schaue dir nun das Spektrogram dieser Audiodateien an. Siehst du Unterschiede im Spektogram zwischen Hunden und Katzen? Achtung, die Samplerate ist nun `16000`.

In [ ]:
...

### Aufgabe 3

Wir erstellen nun ein einfaches Modell, um die Audiodateien zu klassifizieren. Erstelle zuerst eine Funktion welche Eigenschaften aus der Audiodatei extrahiert, wie folgt:
    
1. Öffne die Audiodatei.
2. Berechne den Absolutbetrag der Werte.
3. Berechne dann den durchschnittlichen Wert (also etwa die Lautstärke) und die Varianz (`np.var`).
4. Erkenne ausserdem, ob es sich um einen Hund (Wert `1`) oder um eine Katze (Wert `0`) handelt (aus dem Dateinamen).
5. Der Output sollte sein: `return [mean, variance], is_dog`

In [ ]:
# Input wäre z.B. './data/test/dog_barking_19.wav'
# Resultat wäre z.B. [[1, 2], 0]
def extract_simple_features(file_name):
    ...

Erstelle nun eine Funktion, welche die Trainings- oder Testdaten erstellt. Finde dafür alle Files im richtigen Ordner (also Test oder Train). Führe dann `extract_simple_features` für jedes File aus und sammle die beiden Resultate (Features und Hund/Katze) jeweils in einer Liste.

In [ ]:
# Wenn `train=True`, nimm die Files aus dem `train` Ordner. Sonst die aus dem `test` Ordner.
# Resultat wäre z.B. [[1,2], [3, 4]], [1, 0]
def create_dataset(train=True):
    ...

Erstelle nun einen Random Forest zur Klassifizierung!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

...

 Du kannst übrigens die `score` Funktion (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) verwenden um die Genauigkeit auf den Testdaten zu evaluieren. Wie gut ist die Klassifizierung?

In [ ]:
...

### Aufgabe 4

Wir möchten die Klassifizierung nun noch verbessern, indem wir das Spektogram berücksichtigen. Gehe wie folgt vor:
1. Öffne die Audiodatei.
2. Erstelle das Spektrogram.
3. Errechne den durchschnittlichen Wert pro Frequenz. Wir vernachlässigen also die zeitliche Komponente und schauen nur, wie oft welche Frequenzen vorkommen. Du kannst dafür die `np.mean` Funktion verwenden und vielleicht brauchst du das `axis` Argument. Wenn du es richtig machst müsstest du so ein 129 Zahlen langes Array bekommen.
4. 129 Werte sind noch ein bisschen viele Werte (was könnte passieren, wenn wir die alle verwenden?). Wir kürzen das Array noch, indem wir 3 benachbarte Werte zusammen berücksichtigen (`new = old.reshape(-1, 3).mean(axis=1)`). Nun müsste das Array noch 43 lang sein.
5. Zuletzt normalisieren wir das Array, teile es dafür durch seine Summe. Dies stellt sicher, dass das Spektrum ähnlich aussieht egal ob die Geräsche laut oder leise waren.

In [ ]:
# Resultat wäre z.B. [[1, ..., 2], [3, ...,  4]], [1, 0]
def extract_spectral_features(file_name, samplerate=16000):
    ...

Erstelle nun wieder ein Model mit `extract_spectral_features`. Wie ist die Performance?

### Aufgabe 5

Untersuche nun die Dateien, welche unser Modell falsch klassifiziert. Am einfachsten du erstellst die Features für ein File mit `extract_spectral_features` und machst die Vorhersage, bis du Files findest für welche die Vorhersage falsch ist. Verwende die `predict` Funktion des Modells für die Vorhersage. Tipp: Bei den Hunden findest du schneller falsch klassifizierte.

In [ ]:
...

Hast du eine Idee, warum gewisse Files falsch klassifiziert wurden? Hast du Ideen, wie man das Modell weiter verbessern könnte?